In [1]:
import os
import time

In [2]:
#Size of the file
os.path.getsize("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv")

1968829442

In [3]:
#Read in the data with Dask


In [4]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv")
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.009170293807983398 sec


In [5]:
#Read in the data with Pandas


In [6]:
import pandas as pd
start = time.time()
df = pd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv")
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  26.28349280357361 sec


In [7]:
# Read in the data with Modin and Ray


In [8]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv")
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2021-07-18 04:03:56,897	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


Read csv with modin and ray:  14.974868059158325 sec


In [9]:
# Here Dask is better than Pandas, Modin and Ray, with the least reading time of 0.012 sec


In [10]:

from dask import dataframe as dd
df = dd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv"
,delimiter=',')

In [11]:
df.info()


<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(10), float64(9), int64(5)

In [12]:
#No. of Rows
len(df.index)

12694445

In [13]:
#No, of Columns
len(df.columns)

24

In [14]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [15]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [16]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

In [17]:
#Validation


In [18]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [19]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [20]:

%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - PrimarySubscriberAndThreeOrMoreDependents
      - CoupleAndOneDependent
      - CoupleAndTwoDependents
      - CoupleAndThreeOrMoreDependents
      - RowNumber

Overwriting store.yaml


In [21]:
with open("store.yaml", "r") as ymlfile:
    config_data = (yaml.safe_load(ymlfile))

In [22]:
#data of config file
config_data


{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Rate',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThreeOrMoreDependents - RowNumber']}

In [23]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv"
,delimiter=',')
df_sample.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [24]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/" + config_data['file_name'] + f'.{file_type}'

In [25]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [26]:
(df,config_data)

(          BusinessYear StateCode  IssuerId SourceName  VersionNum  \
 0                 2014        AK     21989       HIOS           6   
 1                 2014        AK     21989       HIOS           6   
 2                 2014        AK     21989       HIOS           6   
 3                 2014        AK     21989       HIOS           6   
 4                 2014        AK     21989       HIOS           6   
 ...                ...       ...       ...        ...         ...   
 12694440          2016        WV     96480      SERFF           2   
 12694441          2016        WV     96480      SERFF           2   
 12694442          2016        WV     96480      SERFF           2   
 12694443          2016        WV     96480      SERFF           2   
 12694444          2016        WV     96480      SERFF           2   
 
                    ImportDate  IssuerId2  FederalTIN RateEffectiveDate  \
 0         2014-03-19 07:06:49      21989  93-0438772        2014-01-01   
 1      

In [27]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')
columns of YAML are: ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThre

In [28]:
if (df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")


col validation passed


In [29]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv("/Users/bhargavaramarajudandu/Desktop/Data Glacier Virtual Internship/File Ingestion/archive/Rate.csv",delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['/Users/bhargavaramarajudandu/Rate.csv.gz/00.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/01.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/02.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/03.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/04.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/05.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/06.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/07.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/08.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/09.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/10.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/11.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/12.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/13.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/14.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/15.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/16.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/17.part',
 '/Users/bhargavaramarajudandu/Rate.csv.gz/18.

In [30]:
#number of files in gz format folder
import os
entries = os.listdir("/Users/bhargavaramarajudandu/Rate.csv.gz/")
for entry in entries:
    print(entry)

26.part
30.part
10.part
06.part
07.part
11.part
27.part
16.part
00.part
20.part
21.part
01.part
17.part
02.part
14.part
18.part
22.part
23.part
19.part
15.part
03.part
08.part
24.part
04.part
12.part
28.part
29.part
13.part
05.part
25.part
09.part


In [31]:
#size of the gz format folder
os.path.getsize("/Users/bhargavaramarajudandu/Rate.csv.gz/")

1056